In [1]:
import pandas as pd
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
from xbrl import XBRLParser
import io
from lxml import etree
import re
import polars as pl
import re

In [17]:
str = 'ffffffffffdddddddddd ffffffdfdfd \n   </ix:resources></ix:header> \ndfdfdhfhd fdfdfd'

r'</ix:resources>\s*</ix:header>'

start_pattern = re.compile(r'</ix:resources>\s*</ix:header>', re.IGNORECASE)
#start_pattern = re.compile(r'.*?</ix:resources>\s*</ix:header>.*?', re.IGNORECASE)
start_match = start_pattern.search(str)
if start_match:
    start_idx = start_match.end()
    print(start_idx)
else:
    print('not found')


64


In [36]:
#content = "<DESCRIPTION>10-Q \n<TEXT><HTML><HEAD> \n<TITLE>10-Q</"
content = '</DIV></Center> \n \n</BODY>         \n          </HTML>      \n            </TEXT>'
#content = 'style="line-height:120%;font-size:10pt;"><span style="font-family:inherit;font-size:10pt;">178</span></div></div>	</div></body>\n</html>\n</XBRL>'

end_pattern = re.compile(r'</html>', re.IGNORECASE)

match = end_pattern.search(content)
if match:
    print("First </html> found at position:", match.start())
else:
    print("No </html> tag found.")

First </html> found at position: 46


In [3]:
def get_random_headers():
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    return headers

In [55]:
link = (
    #"https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks"
    "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
)
df = pd.read_html(link, header=0)[0]

In [21]:
str_df = df.astype(str)
str_df

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
497,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [4]:
report_url = "https://www.sec.gov/Archives/edgar/data/936468/0000936468-20-000016.txt"
response = requests.get(report_url, headers=get_random_headers())

soup = BeautifulSoup(response.text, 'html.parser')

for table in soup.find_all('table'):
    table.decompose()

for table in soup.find_all('h'):
    table.decompose()

for table in soup.find_all('form'):
    table.decompose()


text = soup.get_text()
text[10000:15000]

KeyboardInterrupt: 

In [18]:
report_url = "https://www.sec.gov/Archives/edgar/data/899051/0000899051-20-000029.txt"
response = requests.get(report_url, headers=get_random_headers())

if response.status_code == 200:
        # Response.text contains the content of the .txt file as a string
        content  = response.text
        
        start_pattern = re.compile(r'</ix:resources>\s*</ix:header>', re.IGNORECASE)
        end_pattern = re.compile(r'</html>', re.IGNORECASE)

        start_match = start_pattern.search(content)
        if start_match:
            start_idx = start_match.end()
        else:
            print('fail to find start idx')

        end_match = end_pattern.search(content, start_idx)
        if end_match:
            end_idx = end_match.end()
        else:
            print('fail to find end idx')

        html_content = content[start_idx:end_idx]
        #start_idx = content.find('</ix:resources></ix:header>')
        #print(start_idx)
#
        #if start_idx != -1:
        #    # Adjust the start index to the end of the tag to capture the HTML content correctly
        #    start_idx += len('</ix:resources></ix:header>')
        #print(start_idx)
        #
        #end_idx = content.find('</html>', start_idx)
        #print(end_idx)
#
        #html_content = content[start_idx:end_idx + len('</html>')]

        soup = BeautifulSoup(html_content, 'html.parser')

        for script in soup(["script", "style", "head", "title", "meta", "[document]"]):
            script.decompose() 

        for table in soup.find_all('table'):
            table.decompose()
        
        for a in soup.find_all("a"):
            a.decompose()

        text = soup.get_text()
        print(len(text))
        
        
         
        
else:
    print(f"Error downloading file: {response.status_code}")



177032


In [15]:
print(f'STARTING STR: {text[:100]} \n \n \n ENDING STR: {text[-100:]}')

STARTING STR: UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 10-Q ☒  QUARTERLY REPORT P 
 
 
 ENDING STR: eport to be signed on its behalf by the undersigned thereunto duly authorized.96  www.allstate.com 



In [ ]:
if xbrl_content:
    
    xbrl_data = str(xbrl_content)  # Keep data as string, not bytes
    xbrl_file_like = io.StringIO(xbrl_data)  # Use StringIO for string data
    xbrl_file_like.seek(0)
    

    # Parse the XBRL data using the file-like object
    try:
        xbrl_parser = XBRLParser()
        xbrl = xbrl_parser.parse(xbrl_file_like)
        print(xbrl)
    except Exception as e:
        print("Error parsing XBRL:", e)
else:
    print("XBRL content not found.")

TypeError: 'NoneType' object is not callable

In [78]:
def get_cik():
    link = (
        #"https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks"
        "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    )
    df = pd.read_html(link, header=0)[0]
    snp_str_df = df.astype(str)
    #print(type(str_ciks[0]))
    #cik_set = set(str_df["CIK"])
    #print(snp_str_df)
    snp_str_df['Count'] = snp_str_df.groupby('CIK').cumcount()

    snp_str_df = snp_str_df[snp_str_df['Count'] == 0]

    snp_str_df = snp_str_df.drop(columns=['Count'])

    return snp_str_df

In [79]:
df = get_cik()

In [80]:
set_df = set(df['CIK'])
print(len(df))


if '1564708' in set_df:
    print(True)

ticker = df.loc[df["CIK"] == '1564708', "Symbol"]

print(ticker)

499
True
337    NWSA
Name: Symbol, dtype: object


In [32]:
df[336: 340]

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
336,NEM,Newmont,Materials,Gold,"Denver, Colorado",1969-06-30,1164727,1921
337,NWSA,News Corp (Class A),Communication Services,Publishing,"New York City, New York",2013-08-01,1564708,2013 (News Corporation 1980)
338,NWS,News Corp (Class B),Communication Services,Publishing,"New York City, New York",2015-09-18,1564708,2013 (News Corporation 1980)
339,NEE,NextEra Energy,Utilities,Multi-Utilities,"Juno Beach, Florida",1976-06-30,753308,1984 (1925)


In [73]:
def non_unique_cik(snp_str_df):
    res = snp_str_df["CIK"].value_counts()
    non_unique_ciks = res[res > 1]

    return non_unique_ciks.index


In [75]:
set(non_unique_cik(df))


{'1564708', '1652044', '1754301'}

In [5]:
string = "Name of the Series. Will be used as a column name when used in a DataFrame. When not specified, name is set to an empty string."

In [8]:
ser = pl.DataFrame(string.split())

ser

column_0
str
"""Name"""
"""of"""
"""the"""
"""Series."""
"""Will"""
…
"""set"""
"""to"""
"""an"""
